In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
from scipy.signal import fftconvolve

import bloscpack as bp
import blosc

dm = DataManager(stack='MD592',
                 section=46)

print 'reading image ...',
t = time.time()
dm._load_image(versions=['gray'])
dm._generate_kernels()
print 'done in', time.time() - t, 'seconds'

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2261: DecompressionBombWarning: Image size (167881728 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


reading image ... done in 4.32109379768 seconds


In [5]:
assert dm.ymin-dm.max_kern_size >= 0 and dm.xmin-dm.max_kern_size >= 0 and \
        dm.ymax+1+dm.max_kern_size <= dm.image_height and dm.xmax+1+dm.max_kern_size <= dm.image_width, \
        'Not all pixels within the mask have value from the largest kernel'

def convolve_per_proc(i):
    pf = fftconvolve(dm.image[dm.ymin-dm.max_kern_size : dm.ymax+1+dm.max_kern_size, 
                              dm.xmin-dm.max_kern_size : dm.xmax+1+dm.max_kern_size], 
                     dm.kernels[i], 'same').astype(np.half)
    sys.stderr.write('filtered kernel %d\n'%i)

#     bp.pack_ndarray_file(pf[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size].copy(), 
#                          os.environ['GORDON_RESULT_DIR']+'/feature_%03d.bp'%i)
    
#     return pf[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size]


In [ ]:
feature_0 = pf[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size]
plt.matshow(feature_0);

In [ ]:
# with open('/home/yuncong/CSHL_data_results/feature_000.bp', 'wb') as f:
#     f.write(blosc.pack_array(feature_0))

t = time.time()
# features_0_load = bp.unpack_ndarray_str(bp.pack_ndarray_str(feature_0.copy()))
features_0_load = blosc.unpack_array(blosc.pack_array(feature_0))
print time.time() - t

In [ ]:
# with open('/home/yuncong/CSHL_data_results/feature_000.bp', 'rb') as f:
#     s = f.read()

In [ ]:
t = time.time()
print 'gabor filtering...',

for i in range(dm.n_kernel):
    convolve_per_proc(i)

print 'done in', time.time() - t, 'seconds'

filtered kernel 0
filtered kernel 1
filtered kernel 2
filtered kernel 3
filtered kernel 4
filtered kernel 5


In [ ]:
t = time.time()
print 'gabor filtering...',

# filtered = Parallel(n_jobs=4)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
# features = np.asarray(filtered)
# del filtered

Parallel(n_jobs=4)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))

print 'done in', time.time() - t, 'seconds'

In [ ]:
dm.save_pipeline_result(features, 'features')

In [ ]:
t = time.time()
print 'load filtered values ...',

features = []
for i in range(dm.n_kernel):
    a = bp.unpack_ndarray_file(os.environ['GORDON_RESULT_DIR']+'/feature_%03d.bp'%i).reshape((-1,)) 
    features.append(a)
    
print 'done in', time.time() - t, 'seconds'

In [ ]:
del features

In [ ]:
t = time.time()
print 'load filtered values ...',

features = [bp.unpack_ndarray_file(os.environ['GORDON_RESULT_DIR']+'/feature_%03d.bp'%i).reshape((-1,)) 
                     for i in range(dm.n_kernel)]

print 'done in', time.time() - t, 'seconds'

In [ ]:
del features

In [ ]:
features = features.T

In [ ]:
def rotate_features(fs, j):
    features_tabular = fs.reshape((fs.shape[0], dm.n_freq, dm.n_angle))
    max_angle_indices = features_tabular.max(axis=1).argmax(axis=-1)
    features_rotated = np.reshape([np.roll(features_tabular[i], -ai, axis=-1) 
                               for i, ai in enumerate(max_angle_indices)], (fs.shape[0], dm.n_freq * dm.n_angle))

    del features_tabular, max_angle_indices
    
#     print features_rotated.shape
#     bp.pack_ndarray_file(features_rotated, os.environ['GORDON_RESULT_DIR']+'/featureRotated_batch_%03d.bp'%j)

    return features_rotated

In [ ]:
features.shape[0]

In [ ]:
t = time.time()
print 'rotating features ...',

items_per_job = 100
features_rotated = Parallel(n_jobs=8)(delayed(rotate_features)(fs, i) 
                   for i, fs in enumerate(np.array_split(features, features.shape[0]/items_per_job)))

print 'done in', time.time() - t, 'seconds'

In [ ]:
del features_rotated

In [ ]:
features_rotated = np.vstack(features_rotated)

In [ ]:
features_rotated.shape

In [ ]:
t = time.time()
print 'load filtered values ...',

items_per_job = 100
features_rotated = Parallel(n_jobs=8)(delayed(rotate_features)(fs, i) 
                   for i, fs in enumerate(np.array_split(features[:10000000], 10000000/items_per_job)))
# features_rotated = np.vstack(features_rotated)

print 'done in', time.time() - t, 'seconds'

In [ ]:
features_rotated = np.vstack(features_rotated)

In [ ]:
features_rotated.shape

In [ ]:
from skimage.util import pad

approx_bg_intensity = dm.image[10:20, 10:20].mean()
# approx_bg_intensity = 0

masked_image = dm.image.copy()
masked_image[~dm.mask] = approx_bg_intensity

# padded_image = pad(masked_image, max_kern_size, 'constant', constant_values=approx_bg_intensity)
padded_image = pad(masked_image, dm.max_kern_size, 'linear_ramp', end_values=approx_bg_intensity)

# plt.imshow(padded_image, cm.Greys_r)
# plt.show()

# display(padded_image)

In [ ]:
import time
b = time.time()

# try:
#     features = dm.load_pipeline_result('features', 'npy')
    
# except Exception as e:

def convolve_per_proc(i):
    return fftconvolve(padded_image, dm.kernels[i], 'same').astype(np.half)

padded_filtered = Parallel(n_jobs=16)(delayed(convolve_per_proc)(i) 
                        for i in range(dm.n_kernel))

filtered = [f[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size] for f in padded_filtered]

#     features = np.empty((dm.image_height, dm.image_width, n_kernel), dtype=np.half)
#     for i in range(n_kernel):
#         features[...,i] = filtered[i]

features = np.empty((dm.n_kernel, dm.image_height, dm.image_width), dtype=np.half)
for i in range(dm.n_kernel):
    features[i, ...] = filtered[i]

del filtered

dm.save_pipeline_result(features, 'features', 'npy')

print 'gabor filtering', time.time() - b

In [ ]:
# # visualize a slice of feature responses

# cropped_response = features[-1]
# plt.matshow(cropped_response, cmap=cm.coolwarm)
# plt.colorbar()
# plt.show()

In [ ]:
# from skimage.exposure import rescale_intensity
# cropped_response_vis = rescale_intensity(cropped_response, out_range=(0, 255))
# cropped_response_vis[~dm.mask] = 127
# display(cropped_response_vis.astype(np.uint8))